In [16]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
import pathlib
import random
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib
import statistics
from tqdm import tqdm
from models import bmv_loss
%config Completer.use_jedi = False  # help with autocompletions
%load_ext autoreload
%autoreload 2
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Num GPUs Available:  1


In [17]:
dev_mode = False
batch_size = 16
pre_encode_size = 1024
projection_units = 128
EPOCHS = 1  # can be also 40-45  # try also 0 and 1.
model_name = f'model_{batch_size}_{EPOCHS}.h5'

In [18]:

class batch_generator:
    def __init__(self, batch_size=batch_size):
        self.batch_size = batch_size
        self.data = []
        self.batch = []
        self.image_size = [160, 160]
        self.crop_size = [120, 120, 3]
    def create_img_list(self):
        self.luna_dir = './train/'
        self.img_list = []
        for img in os.listdir(self.luna_dir):
            full_path = os.path.join(self.luna_dir, img)
            self.img_list.append(full_path)

    def load_luna(self):
        self.samples = []
        i = 0
        for img in self.img_list:
            i +=1
            if dev_mode:
                if i > 100:
                    break
            loaded_img = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(img))
            self.samples.append(loaded_img)
            # i need to use the real image and one augmented img and than add batch_size -2 augmented images
            # from different source images
    
    def gen_batch_2(self):
        num_of_batches = len(self.samples)//self.batch_size  # e.g 100//15 = 6
        epoch_mv = []

        for i in range(num_of_batches):  # e.g: 6
            batch_1 = []
            batch_2 = []
            for j in range(self.batch_size):  #15
                batch_1.append(self.get_tensor_aug_img(self.samples[i]))
                batch_2.append(self.get_tensor_aug_img(self.samples[i]))
            mv_batch = [batch_1, batch_2]
            mv_batch = np.array(mv_batch)

            epoch_mv.append(mv_batch)
            
        return(epoch_mv)
    

    def get_tensor_aug_img(self, img):
        source_img = tf.image.resize(img, self.image_size)
        self.batch.append(tf.convert_to_tensor(source_img)) 
        aug_img = tf.image.random_flip_left_right(tf.image.random_crop(value=img, size= self.crop_size))
        aug_img = tf.image.resize(aug_img, self.image_size)
        return aug_img

In [19]:

# Generate batches
gen = batch_generator()
gen.create_img_list()
gen.load_luna()
epoch = gen.gen_batch_2()  # 1000//batch*batch*2*160*160*3


In [20]:
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)
IMG_SHAPE

(160, 160, 3)

In [6]:
# def create_model():
#     # Input size is images size
#     input_size = (160, 160, 3) 
#     # projection units size was suggested by the paper author
    
#     base_model = tf.keras.applications.MobileNetV2(input_shape=input_size,
#                                                    include_top=False,
#                                                    weights='imagenet')
#     # freeze the base model
#     base_model.trainable = False
#     inputs = keras.Input(shape=input_size)
#     norm_input = tf.keras.layers.experimental.preprocessing.Normalization( axis =-1)(inputs)
#     features = base_model(norm_input)
#     flat = layers.Flatten()(features)
#     pre_encode = layers.Dense(pre_encode_size)(flat)
#     norm_encode = tf.math.l2_normalize(pre_encode)
#     encode = layers.Dense(projection_units, activation="relu")(norm_encode)
#     outputs = tf.math.l2_normalize(encode)
#     model = keras.Model(inputs=inputs, outputs=outputs, name="mobileNetWithProjection")
#     return model

In [7]:
from models import *

In [8]:
# Creating model
self_trans_model = create_contrastive_model(pre_encode_size, projection_units)

In [9]:
for layer in self_trans_model.layers:
    print(layer.trainable, layer, type(layer))

self_trans_model.layers[2].trainable

True <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f729c3b8690> <class 'tensorflow.python.keras.engine.input_layer.InputLayer'>
True <tensorflow.python.keras.layers.preprocessing.normalization.Normalization object at 0x7f729c321f90> <class 'tensorflow.python.keras.layers.preprocessing.normalization.Normalization'>
False <tensorflow.python.keras.engine.functional.Functional object at 0x7f729c34d790> <class 'tensorflow.python.keras.engine.functional.Functional'>
True <tensorflow.python.keras.layers.core.Flatten object at 0x7f729c339dd0> <class 'tensorflow.python.keras.layers.core.Flatten'>
True <tensorflow.python.keras.layers.core.Dense object at 0x7f729c22ee50> <class 'tensorflow.python.keras.layers.core.Dense'>
True <tensorflow.python.keras.layers.core.TFOpLambda object at 0x7f729c355e90> <class 'tensorflow.python.keras.layers.core.TFOpLambda'>
True <tensorflow.python.keras.layers.core.Dense object at 0x7f729c232f90> <class 'tensorflow.python.keras.layers.core.Den

False

In [10]:
# dir(self_trans_model.layers[2].layers)
for layer in self_trans_model.layers[2].layers:
    print(layer.trainable, layer)

False <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f73ac426a50>
False <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73ac426910>
False <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f73abdd4f90>
False <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f73abeceb10>
False <tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7f73a0095f50>
False <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f73a0046e50>
False <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f73a005fdd0>
False <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73a0046e10>
False <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f73a0069990>
False <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73a0058e50>
False <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 

In [11]:
self_trans_model.layers

In [12]:
import gc


In [13]:
# Track Results
train_loss_results = []
train_accuracy_results =[]

# Loop variables

# Training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
print ("Train started:")

for e in range(EPOCHS):
    epoch_loss = []
    print("starting epoch")
#     epoch = gen.gen_batch_2()
    for idx in range(len(epoch)):
        with tf.GradientTape() as tape:
            logits_1 = self_trans_model(epoch[idx][0],training =True)
            logits_2 = self_trans_model(epoch[idx][1],training =True)
            mvb_loss = bmv_loss(logits_1, logits_2, 0.1)
            grads = tape.gradient(mvb_loss, self_trans_model.trainable_weights)
            optimizer.apply_gradients(zip(grads, self_trans_model.trainable_weights))
        if not idx% 10:
            print(mvb_loss.numpy())

        epoch_loss.append(mvb_loss.numpy())
    avrg_loss = statistics.mean(epoch_loss)
    print(f"Epoch Num {e}, Avrg Loss: {avrg_loss}")
    print(gc.collect())
print("success")


Train started:
starting epoch
43.33409
43.528687
43.349667
43.334602
43.36657
43.34536
43.347824
Epoch Num 0, Avrg Loss: 43.365753173828125
0
success


In [14]:
self_trans_model.save(model_name)

loaded = tf.keras.models.load_model(model_name)

In [15]:
model_name

'model_16_1.h5'